In [1]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$Y \sim \mathcal{N}(np \, np\cdot(1 - p))\$$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для средней конверсии пользователей из контрольной выборки с уровнем значимости 95%. Запишите значения левой и правой границ через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой

#### РЕШЕНИЕ

In [2]:
df = pd.read_csv("a_b_testing.csv")
df_A = df[df['group']=='A']
n = df_A.shape[0]
print(n)
p = df_A['converted'].mean()
p

2037


0.4118802160039273

In [3]:
alpha = 1-0.95
alpha

0.050000000000000044

In [4]:
z_value = stats.norm.ppf(q = 1-alpha/2)
z_value

1.959963984540054

In [5]:
interval = z_value * np.sqrt(p*(1-p)/n)
interval

0.02137326827255512

In [6]:
lb = p - interval
lb

0.3905069477313722

In [7]:
ub = p + interval
ub

0.43325348427648247

In [8]:
answer1 = "{:.2f},{:.2f}".format(lb,ub)
answer1

'0.39,0.43'

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Задание по статистике Стьюдента 

Найдите значение статистики Стьюдента в предположении независимости выборок

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

`n` - размер контрольной выборки, `m`  - размер тестовой выборки

Ответ запишите в переменную `answer2` с точностью до 2 знака после запятой

### РЕШЕНИЕ

In [9]:
df = pd.read_csv('a_b_testing.csv')
df_A = df[df['group']=='A']

# Размерность контрольной выборки 
n = df_A.shape[0]
df_B = df[df['group']=='B']

# Размерность тестовой выборки 
m = df_B.shape[0]
print(n, m)

# Средние по выборкам
pA = df_A['converted'].mean()
pB = df_B['converted'].mean()
pA, pB

2037 1963


(0.4118802160039273, 0.4370860927152318)

In [10]:
# Дисперсия по генеральной выборке
st_dev_A = df_A.std()
st_dev_B = df_B.std()
st_dev_A, st_dev_B

(converted    0.492295
 dtype: float64, converted    0.496152
 dtype: float64)

In [11]:
t = (pA - pB)/(st_dev_A * st_dev_A / n + st_dev_B * st_dev_B / m) ** 0.5
answer2 = "{:.2f}".format(t[0])
answer2

'-1.61'

### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [12]:
from scipy.stats import ttest_ind
df = pd.read_csv('a_b_testing.csv')

In [13]:
# Размерность контрольной выборки 
df_A = df[df['group']=='A']
n = df_A.shape[0]
df_B = df[df['group']=='B']

# Размерность тестовой выборки 
m = df_B.shape[0]
print(n, m)

2037 1963


In [14]:
t = stats.ttest_ind(df_A['converted'].sample(n), df_B['converted'].sample(m))
answer3 = "{:.2f}".format(t[1])
answer3

'0.11'

Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой

### Ответы

In [15]:
output = """Confidence interval:{0}
T score custom {1:.2f}
p value {2:.2f}"""
print(output.format(answer1, answer2, answer3))

ValueError: Unknown format code 'f' for object of type 'str'